In [5]:
# Load model directly
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

torch.manual_seed(17)

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-small", use_fast=True, 
                                          add_prefix_space=True)
model = AutoModelForCausalLM.from_pretrained("prajjwal1/bert-small", return_dict_in_generate=True, 
                                             pad_token_id=tokenizer.eos_token_id).to(device)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
test_seq = (["Hi","test", "test"])
tokenizer.convert_tokens_to_ids(test_seq)

[100, 3231, 3231]

In [11]:
vocab = tokenizer.get_vocab()
len(vocab)

30522

In [15]:
#get a list of all the tokens in the vocabulary
tokens = list(vocab.keys())
# get the index of the token "test" in the vocabulary
test_token_index = tokens.index("test")
# get the index of the token "test" in the vocabulary
print(test_token_index)
print(tokens[test_token_index])

28369
test


In [94]:
prompt = " a   "
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
print(input_ids)

with torch.no_grad():
    output = model(input_ids)
    logits = output[0]
    probs = logits.softmax(-1)

print(probs.shape)

tensor([[ 101, 1037,  102]])
torch.Size([1, 3, 30522])


In [74]:
for i in range(probs.shape[-1]):
    prob = probs[0, -1, i].item()
    # if prob > 0.0001:
    print(f"Probability of {tokenizer.decode(i)} ({i}) is {prob:.4f}")

Probability of [PAD] (0) is 0.0000
Probability of [unused0] (1) is 0.0000
Probability of [unused1] (2) is 0.0000
Probability of [unused2] (3) is 0.0000
Probability of [unused3] (4) is 0.0000
Probability of [unused4] (5) is 0.0000
Probability of [unused5] (6) is 0.0000
Probability of [unused6] (7) is 0.0000
Probability of [unused7] (8) is 0.0000
Probability of [unused8] (9) is 0.0000
Probability of [unused9] (10) is 0.0000
Probability of [unused10] (11) is 0.0000
Probability of [unused11] (12) is 0.0000
Probability of [unused12] (13) is 0.0000
Probability of [unused13] (14) is 0.0000
Probability of [unused14] (15) is 0.0000
Probability of [unused15] (16) is 0.0000
Probability of [unused16] (17) is 0.0000
Probability of [unused17] (18) is 0.0000
Probability of [unused18] (19) is 0.0000
Probability of [unused19] (20) is 0.0000
Probability of [unused20] (21) is 0.0000
Probability of [unused21] (22) is 0.0000
Probability of [unused22] (23) is 0.0000
Probability of [unused23] (24) is 0.0000


In [86]:
words = ["medicine", "science"]
indexes = [tokenizer.encode(word) for word in words]

print(indexes)

for i in indexes:
    word_prob = torch.prod(torch.stack([probs[0, -1, idx] for idx in i]))
    print(f"Probability of {tokenizer.decode(i)} is {word_prob:.20f}")


[[101, 4200, 102], [101, 2671, 102]]
Probability of [CLS] medicine [SEP] is 0.00000000000000000460
Probability of [CLS] science [SEP] is 0.00000000000000002013


In [87]:
words_2 = ["medicine", "science"]
indexes = [tokenizer.encode(word) for word in words]
#removing the cls and sep tokens
indexes = [i[1:-1] for i in indexes]
print(indexes)

for i in indexes:
    word_prob = torch.prod(torch.stack([probs[0, -1, idx] for idx in i]))
    print(f"Probability of {tokenizer.decode(i)} is {word_prob:.20f}")


[[4200], [2671]]
Probability of medicine is 0.00000477175490232185
Probability of science is 0.00002085873165924568


If we remove the CLS and SEP token from the input the probability 

In [88]:
words_probs = {}

for i in range(probs.shape[-1]):
    prob = probs[0, -1, i].item()
    words_probs[(tokenizer.decode(i)).replace(" ", "")] = prob


print(words_probs)
#get the top 5 words with the highest probability
top_words = sorted(words_probs, key=words_probs.get, reverse=True)[:5]
print(top_words)

print(words_probs["medicine"])
print(words_probs["science"])


{'[PAD]': 9.423867908253669e-08, '[unused0]': 1.1952600686981896e-07, '[unused1]': 4.1213493773284426e-07, '[unused2]': 5.197745167606627e-07, '[unused3]': 2.0293026636863942e-07, '[unused4]': 2.874842266464839e-07, '[unused5]': 5.662199100697762e-07, '[unused6]': 1.1873957816987968e-07, '[unused7]': 3.8850978967275296e-07, '[unused8]': 3.562834649528668e-07, '[unused9]': 1.1913635233895548e-07, '[unused10]': 3.906591246050084e-07, '[unused11]': 1.2890804157450475e-07, '[unused12]': 3.0248111215769313e-07, '[unused13]': 6.08823597758601e-07, '[unused14]': 1.5831663802146068e-07, '[unused15]': 7.657740752620157e-07, '[unused16]': 3.89130519806713e-07, '[unused17]': 2.7917764100493514e-07, '[unused18]': 1.5687682264342584e-07, '[unused19]': 1.3884393013086083e-07, '[unused20]': 2.321530246263137e-06, '[unused21]': 2.510415697543067e-07, '[unused22]': 1.5643801987152983e-07, '[unused23]': 4.953610641678097e-07, '[unused24]': 8.447223081020638e-07, '[unused25]': 1.4379085655491508e-07, '[u

In [85]:
wanted_words_probs = {}
for word in words:
    wanted_words_probs[word] = words_probs[word]

total = sum(wanted_words_probs.values())
for word in wanted_words_probs:
    wanted_words_probs[word] = wanted_words_probs[word] / total

print(wanted_words_probs)

{'medicine': 0.1861749635871919, 'science': 0.8138250364128081}
